# The Newton-Raphson Method

## **Prelude**

If you enter $\sqrt{2}$ into a calculator, you'll probably see an output similar to $1.4142135623730950$. How does your calculator find this value?

We're going to look at two methods today for solving for the zeros of a function. That is to say, if we have a (nice) function $f(x)$, we're going to develop methods that we can use to approximate input values $x=c$ where $f(c)=0$.

The first method we'll look at, the Bisection Method, requires only that $f(x)$ is a continuous function. We'll repeatedly use the Intermediate Value Theorem to find closer and closer approximations to the zeros of a function, by analyzing the behavior of the function on smaller and smaller intervals.

The second method we'll look at, the Newton-Raphson Method, requires that $f(x)$ is a differentiable function. This is a stronger requirement and we'll see that, in-turn, this method often allows for "faster" approximation of zeros. Often times, this method is how your calculator will find the square root of a number.

## **Learning Goals**

In this notebook, we're going to do the following:

1. We will learn two methods for approximating the zeros of nice functions (the Bisection Method and the Newton-Raphson Method) using the concepts that we've learned from calculus so far.
2. We will analyze the performance of both methods, using the tools from Python that we've learned so far.

# 1. The Bisection Method and the Intermediate Value Theorem

Let's start with an example, say $f(x)=x^3+x-1$. Because this is an odd degree polynomial, we have

$$\lim_{x\rightarrow -\infty} f(x)=-\infty \quad \mbox{and} \quad \lim_{x\rightarrow \infty} f(x) =\infty.$$

Since we know that polynomials are continuous, the change in sign tells us that our function crosses the $x$-axis at some point. We can get an initial guess for where this happens by plotting the graph on a large enough interval.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def f(x):
  return x**3+x-1

x_inputs=[]
y_outputs=[]

step=0.01
for i in range(400):
  x_inputs.append(-2+i*step)
  y_outputs.append(f(x_inputs[i]))

plt.grid(True) # This function is new, it adds a grid to our plot
plt.axhline(0, color='black', linewidth=0.8) # This function is also new, it
# adds a horizontal line at 0 to our plot
plt.plot(x_inputs,y_outputs)

It looks like there is a zero between $x=0.5$ and $x=1.0$. We can confirm this by checking these values, and appealing to the Intermediate Value Theorem.

In [ ]:
f(0.5)

In [ ]:
f(1.0)

Because the function is continuous and changes sign on the interval $[0.5,1]$, the Intermediate Value Theorem guarantees that there is a zero in this interval. Let's call this value $x=c$. Our goal is to get a good approximation for $c$.

Since we don't know where the zero is in this interval, we may as well guess that it is in the middle. Therefore, an approximation to the zero is

$$x_1=0.5+(1-0.5)/2=0.75.$$

What is the *error* in our approximation? Well, the true value of $c$ could be very close to either $0.5$ or $1.0$. So, if we call $\epsilon_1$ our error, then we only know:

$$\epsilon_1 = |x_1-c|\leq |x_1-0.5|=|x_1-1.0|=0.25.$$

To get a better approximation, with a smaller possible error, we can check the value of $f$ at $0.75$, and repeat this process for one of the resulting interval halves. At this point, let's make it more automated.

The following will be our set-up.

In [ ]:
# We'll make a list that contains the endpoints of our interval
I_1=[0.5, 1.0]

# We'll make a function that acts like f
def f(x):
  return x**3+x-1

# For convenience, let's also make this function:
def sign(x):
  if x == 0:
    return 0
  if x != 0:
    return abs(x)/x # abs(x) is just another name for |x|

def half_length(a,b):
  return (b-a)/2

# We'll make a function that takes in two numbers a,b
# with a<b and returns the midpoint
def midpoint(a,b):
  return a+half_length(a,b)

# We'll make a function that produces the next interval from our
# previous one
def bisect(I):
  a=I[0]
  b=I[1]

  c=midpoint(a,b)

  if sign(f(a)) == sign(f(c)):
    I=[c,b]
    return I

  if sign(f(b)) == sign(f(c)):
    I=[a,c]
    return I

  if sign(f(c)) == 0: # Just in case
    return [c,c]

We can now create a sequence of approximations to our root, by iterating with a `for` loop.

In [ ]:
x_approx = []
I = [0.5,1.0]

for i in range(10):
  x_approx.append(midpoint(I[0],I[1]))
  I=bisect(I)

print(x_approx)

Let's keep track of the upper endpoints of our intervals, the lower endpoints of our intervals, and the maximum possible error that we could have had too.

In [ ]:
x_approx = []
upper=[]
lower=[]
error=[]

I = [0.5,1.0]

for i in range(10):
  upper.append(I[1])
  lower.append(I[0])
  error.append(half_length(I[0],I[1]))
  x_approx.append(midpoint(I[0],I[1]))
  I=bisect(I)

print("Upper endpoints:", upper)
print("Midpoints:", x_approx)
print("Lower endpoints:", lower)

print("Errors:", error)

We know how to plot this data, after the last Jupyter Notebook assignment.

In [ ]:
n=[]
for i in range(10):
  n.append(i)

plt.title("Upper endpoint, midpoint, and lower endpoints")
plt.plot(n,upper)
plt.scatter(n,upper)
plt.plot(n,x_approx, color='r')
plt.scatter(n,x_approx, color='r')
plt.scatter(n,lower, color='g')
plt.plot(n,lower, color='g')

The error is half of the distance between the upper and lower endpoints. If we plot the list of errors, we should see a converging sequence.

In [ ]:
plt.title("Sequence of errors")
plt.grid(True)
plt.axhline(0, color='black', linewidth=0.8)
plt.scatter(n,error, color='r')

To confirm one more time, we can explicitly see what happens when we evaluate our function $f(x)$ to the list of approximations $x_{approx}$.

In [ ]:
for i in range(10):
  print(f(x_approx[i]))

To summarize, here is the general algorithm for approximating a solution $x=c$ to $f(x)=0$ for a continuous function $f(x)$ using this method.

**The Bisection Method**

1. Find an interval $I_1=[a,b]$ with $a<b$ where $f(x)$ has a solution $x=c$.
2. Calculate the midpoint of $[a,b]$. This will be your first approximation $x_1$ to the solution $x=c$.
3. Check which of the intervals $[a,x_1]$ or $[x_1,b]$ contain $x=c$. This will be your new interval $I_2$.
4. Repeat, from step 2, with the interval $I_2$ to get an approximation $x_2$ and an interval $I_3$. Continue as many times as needed.

## 1. Exercises

**Exercise 1**

Use the bisection method to compute $\sqrt{2}$ with an error of $0.0001$. You should use the function $f(x)=x^2-2$ to do this, with starting interval $I_1=[1,2]$.

Answer these questions:

a) What is your final approximation and how many iterations did it take to get your approximation?

b) Let $\{\epsilon_i\}_{i=1}^\infty$ be the sequence of maximal possible errors for each approximation. What kind of sequence is this? Does this sequence converge?

**Exercise 2**

Make two plots that include the following information:

a) One plot should include the upper endpoint, lower endpoint, and midpoints for each interval that you create using the bisection method in Exercise 1.

b) The second plot should graph the squares your approximations. This should be a sequence with limit at $y=2$.

**Exercise 3**

Use the bisection method to compute $\sqrt{5}$ with an error at most $0.0001$.

# 2. Derivatives and the Newton-Raphson Method

We're going to continue to look at the solution to the polynomial $f(x)=x^3+x-1$. This time, instead of using the Bisection Method, we'll use a method called *the Newton-Raphson Method*.

Our goal is still to find an approximation of the value $x=c$ in the interval $[0.5,1]$ that satisfies $f(c)=0$. The main new idea of this method is to use a "linear approximation" to our function at a point near $x=c$.

**The Newton-Raphson Method**
1. We'll pick a random point $x_1=a_1$ with $0.5\leq a_1\leq 1$. This will give us an approximation to the zero of $f(x)$.
2. We'll write down the tangent line to $y=f(x)$ at $x_1$. This will give us an approximation to $f(x)$.
3. Then we find where the tangent line meets the $x$-axis. This will give us an approximation to $x=c$. We'll call this point where the tangent line meets the $x$-axis $x_2$.
4. We'll repeat starting with step 2, until we are close enough.

Let's walk through just one step in the process first. We'll start with $x_1=1$, and try to find $x_2$.

In [ ]:
# We define our function again so that it's nearby
def f(x):
  return x**3+x-1

# We'll need the derivative of f(x)
def df(x):
  return 3*x**2+1

# The tangent line at a is y-f(a)=f'(a)(x-a).
# We want to know what x value gives y=0.
# Solve for this value and compare with the function below.
def tangent_line_intercept(a):
  return -f(a)/df(a) + a

print(tangent_line_intercept(1))

Thus, $x_2=0.75$. It might be nice to see this in a picture.

In [ ]:
def tangent_line(a,x):
  return df(a)*(x-a)+f(a)

x_inputs=[]
y_outputs=[]
tl_outputs=[]

step=0.01
for i in range(80):
  x_inputs.append(0.5+i*step)
  y_outputs.append(f(x_inputs[i]))
  tl_outputs.append(tangent_line(1,x_inputs[i]))

plt.grid(True)
plt.axhline(0, color='black', linewidth=0.8)

plt.plot(x_inputs,tl_outputs)
plt.plot(x_inputs,y_outputs)

Our approximation going from $x_1=1$ to $x_2=0.75$ certainly seems to have brought us closer to the zero $x=c$. Let's automate this process and see what happens after a few iterations.

In [ ]:
x_approx=[1]

for i in range(10):
  x_approx.append(tangent_line_intercept(x_approx[i]))

print(x_approx)


Notice that the sequence of approximations stabilizes (stops changing) after only a few iterations. Compare this to the example above using the Bisection Method. How should we check if our sequence has converged to something useful? Let's check the value of $f(x)$ at the last approximation.

In [ ]:
f(x_approx[10])

Probably you'll see a number which isn't quite $0$ (on my screen I see, 0.000000000000000111022...). But, this is probably the finest approximation to $x=c$ that we'll be able to get with the tools that we're currently using (i.e. floating point decimal values that are native to computer hardware).

Let's take the last value of our sequence $x_{approx}[10]$, one of the stabilized terms, as the true value for $x=c$ and plot the error terms for this method. We can calculate the "exact" error in this case.

In [ ]:
n=[]
error=[]
for i in range(11):
  n.append(i)
  error.append(x_approx[i]-x_approx[10])

plt.grid(True)
plt.title("Sequence of errors")
plt.scatter(n,error, color='r')

So which of the two methods is better? Weeeeeeeell, it depends.

When it works, the Newton-Raphson method will often converge much faster than the Bisection method. Unfortunately, a bad initial guess for the Newton-Raphson method will cause problems (e.g. choosing too coarse of a first guess may lead to an approximation of the wrong zero, or to a sequence that doesn't converge. See the next example of approximating solutions to $g(x)=x^4-3x^2-2=0$).

In [ ]:
def g(x):
  return x**4-3*x**2-2

def dg(x):
  return 4*x**3-6*x

def tangent_line_g(a,x):
  return dg(a)*(x-a)+g(a)

def tangent_line_g_intercept(a):
  return -g(a)/dg(a)+a

x_inputs=[]
y_outputs=[]
tangent_1=[]
tangent_2=[]
tangent_3=[]

t1=1
t2=tangent_line_g_intercept(t1)
t3=tangent_line_g_intercept(t2)
print("t2 is:", t2)
print("t3 is:", t3)

step=0.01
for i in range(400):
  x_inputs.append(-2+i*step)
  y_outputs.append(g(x_inputs[i]))
  tangent_1.append(tangent_line_g(t1,x_inputs[i]))
  tangent_2.append(tangent_line_g(t2,x_inputs[i]))

plt.grid(True)
plt.ylim(-5,2)
plt.axhline(0,color='black', linewidth=0.8)
plt.plot(x_inputs,y_outputs, color='black')
plt.plot(x_inputs,tangent_1, color='r')
plt.plot(x_inputs,tangent_2, color='blue')

In this example, the sequence we'd generate using the Newton-Raphson method with initial guess $x_1=1$ is $x_2=-1$, $x_3=1$, $x_4=-1$,..., $x_n=(-1)^{n+1}$. We know this sequence doesn't converge, since it alternates infinitely often between 1 and -1.

## 2. Exercises

**Exercise 4**

Use the Newton-Raphson method to compute $\sqrt{2}$ with an error of $0.0001$. You should use the function $f(x)=x^2-2$ to do this, with starting approximation $x_1=2$.

Answer these questions:

a) What is your final approximation and how many iterations did it take to get your approximation?

b) Did it take less iterations or more iterations to reach your final approximation compared to using the Bisection method?

**Exercise 5**

Use the Newton-Raphson method to compute an approximation to $\sqrt[3]{13}$. Make sure your approximation has an error at most $0.0001$ (this means the 4th decimal place should stabilize in your approximations).

**Exercise 6**

Use the Newton-Raphson method to find an approximate solution to the equation $e^x-x=2$. Your solution should be accurate with an error at most $0.0001$.